In [ ]:
# connect to google colab
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# base path
DATA_PATH = './drive/MyDrive/fyp-code/codes/data/emotion_intensity/'

In [ ]:
# usual import
import pandas as pd
import numpy as np
from tqdm import tqdm
# for the api calls
import requests
import os
import re

In [ ]:
# to call the file that contains the api keys
base_folder = '/content/drive/MyDrive/fyp-code/codes'
training_path = os.path.join(base_folder, "sentic_api_key.zip") 
!unzip $training_path

#from sentic_api_key import CONCEPT_PARSING_KEY, POLARITY_CLASSIFICATION_KEY, INTENSITY_RANKING_KEY, EMOTION_CATEGORIZATION_KEY, DEPRESSION_IDENTIFICATION_KEY
from sentic_api_key import DEPRESSION_IDENTIFICATION_KEY

Archive:  /content/drive/MyDrive/fyp-code/codes/sentic_api_key.zip
  inflating: sentic_api_key.py       


## Import Short Text

In [ ]:
# import the short dataset
short_data = pd.read_csv(DATA_PATH+'emotion_intensity_depressed_clean_short_data_vader_t2e.csv')[['text_cleaned_t2e_vader', 'Label']]
short_data.head(3)

,text_cleaned_t2e_vader,Label
0,I get to spend New Year is home again alone an...,1
1,"Depressed and lonely Stuck in a deep, never en...",1
2,Learning to pretend to have a good time had be...,1


In [ ]:
# sample sentences
print(short_data.text_cleaned_t2e_vader[0])
print(short_data.text_cleaned_t2e_vader[15])

I get to spend New Year is home again alone and lonely. ??
Absolutly shocking service from especially when owed Expected better from a comp Ive dealt with yr unhappy badservice


## Import Long Text

In [ ]:
# import the long dataset
long_data = pd.read_csv(DATA_PATH+'emotion_intensity_depressed_clean_long_data_vader_t2e.csv')[['text_cleaned_t2e_vader', 'Label']]
long_data.head(3)

,text_cleaned_t2e_vader,Label
0,Just another night. Another night of feeling l...,1
1,Is it possible to fake depression? I have been...,1
2,Imagine being attractive Imagine what it would...,1


In [ ]:
# sample sentences
print(long_data.text_cleaned_t2e_vader[0])
print(long_data.text_cleaned_t2e_vader[15])

Just another night. Another night of feeling lonely and just wondering what I did wrong in life to deserve this unhappiness. I have never felt a pain stronger than being rejected by the love of your life. The person who give you a purpose. The person who is supposed to make everything better. You would give your life for this person and they just do not love you anywhere near the same.
My cat just got hit by a car I cannot even think about it too much. My family is a little sad but are kinda telling me to get over it. I have been in such a terrible depressive spell for the last few week and this just ruined everything. When older pet die by natural cause I do not cry a much a because I know we gave them the best life and they lived very long but my suki wa only year old she wa so fat and adorable and I cannot believe she is gone just because of a bad driver. She always came to me when I wa upset she would lay on top of me and snuggle. I wish I had more time with her. now I have almost 

## Do a test on the API call first

In [ ]:
# test sentence
response = requests.get(f"https://sentic.net/api/en/{DEPRESSION_IDENTIFICATION_KEY}.py?text=a little sad and happy somehow")
print(response.status_code)
# the values received as the response, usually is in json, but this is in string
print(response.text)
# check the type of the response
print(type(response.text))

200
33%

<class 'str'>


In [ ]:
# text response return as string because of the percentage, need to convert it to a float and normalized to 0 and 1
score = int(re.sub("[^0-9]", "", response.text))*0.01
score

0.33

## Helper function to get the depression scores from the sentic API

In [ ]:
def get_depression_score_from_api(text):
    response = requests.get(f"https://sentic.net/api/en/{DEPRESSION_IDENTIFICATION_KEY}.py?text={text}")
    # check status code if success or not
    if response.status_code == 200:
        score = float(re.sub("[^0-9.]", "", response.text))*0.01
    else:
        print("status code not 200!")
        score = -1 # negative one to denote that an error occurred when processing this piece of text
    
    return score

## Apply the Depression Identification API into the text data we have

In [ ]:
# try on one piece of data
score = get_depression_score_from_api(short_data.text_cleaned_t2e_vader[10])
score

0.466

### Short data

In [ ]:
# do it for the whole dataframe
# create a list to store the depression score for the short text
short_data_depression_score_list = []

# get the depression score for the whole short text dataset
for sentence in tqdm(short_data.text_cleaned_t2e_vader.tolist()):
    score = get_depression_score_from_api(sentence)
    short_data_depression_score_list.append(score)

  5%|▍         | 38/834 [00:43<14:31,  1.09s/it]

status code not 200!


  5%|▌         | 42/834 [00:47<14:21,  1.09s/it]

status code not 200!


  6%|▌         | 51/834 [00:57<14:25,  1.11s/it]

...

 92%|█████████▏| 771/834 [14:11<01:08,  1.09s/it]

status code not 200!


 98%|█████████▊| 820/834 [15:05<00:15,  1.12s/it]

status code not 200!


100%|██████████| 834/834 [15:21<00:00,  1.10s/it]


In [ ]:
# create a dataframe to store the depression scoring
depression_score_api_short_df = pd.DataFrame()
depression_score_api_short_df['text_cleaned'] = short_data.text_cleaned_t2e_vader
depression_score_api_short_df['depression_score'] = short_data_depression_score_list
depression_score_api_short_df.head(50)

,text_cleaned,depression_score
0,I get to spend New Year is home again alone an...,0.44333
1,"Depressed and lonely Stuck in a deep, never en...",0.80000
2,Learning to pretend to have a good time had be...,0.11000
3,So far he stop texting me after I said somethi...,0.25000
4,sigh ?? I have not cried so much I am in so mu...,0.44333
5,Thank god the last presentation is over! tomor...,0.00000
6,"No, I am not wouldepressed because of the weat...",0.66667
7,Very sad day in Edmonton! Praying for all who ...,1.00000
8,Losing my uncle after the wedding it just..I d...,0.33333
9,Nobody care in real life and nobody care here....,0.42571


In [ ]:
# look at the rows with a depression score with -1 and do the analysis again because of the status code error just now
for index, sentence in tqdm(enumerate(depression_score_api_short_df.text_cleaned.tolist())):
    if depression_score_api_short_df.depression_score[index] == -1.0000:
        score = get_depression_score_from_api(sentence)
        depression_score_api_short_df.depression_score[index] = score

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
38it [00:01, 25.01it/s]

status code not 200!


42it [00:02, 13.49it/s]

status code not 200!


51it [00:04, 10.43it/s]

...

status code not 200!


771it [00:36, 20.93it/s]

status code not 200!


834it [00:37, 22.09it/s]

status code not 200!


### Long data

In [ ]:
# create a list to store the depression score for the long text
long_data_depression_score_list = []

# get the depression score for the whole short text dataset
for sentence in tqdm(long_data.text_cleaned_t2e_vader.tolist()):
    score = get_depression_score_from_api(sentence)
    long_data_depression_score_list.append(score)

  1%|▏         | 19/1436 [00:25<27:17,  1.16s/it]

status code not 200!


  7%|▋         | 95/1436 [02:12<35:46,  1.60s/it]

status code not 200!


 10%|█         | 149/1436 [03:28<30:33,  1.42s/it]

...

 98%|█████████▊| 1409/1436 [33:17<00:32,  1.22s/it]

status code not 200!


 99%|█████████▉| 1426/1436 [33:40<00:13,  1.38s/it]

status code not 200!


100%|██████████| 1436/1436 [33:54<00:00,  1.42s/it]


In [ ]:
# create a dataframe to store the depression scoring
depression_score_api_long_df = pd.DataFrame()
depression_score_api_long_df['text_cleaned'] = long_data.text_cleaned_t2e_vader
depression_score_api_long_df['depression_score'] = long_data_depression_score_list
depression_score_api_long_df.head(50)

,text_cleaned,depression_score
0,Just another night. Another night of feeling l...,0.37600
1,Is it possible to fake depression? I have been...,0.47260
2,Imagine being attractive Imagine what it would...,0.66000
3,"Best moment to have anxiety It is am, I am tir...",0.16298
4,"hi, I am a year-old male from the uk, over the...",0.51067
5,I do not want to die feeling like I wa never l...,0.38480
6,I wa just curious what your feeling are on lon...,0.47901
7,"My hearing, attentiveness and social awareness...",0.15250
8,Hello out there.. Ugh. I battle with depressio...,0.38806
9,Currently waiting at the hospital to be seen I...,0.31188


In [ ]:
# look at the rows with a depression score with -1 and do the analysis again because of the status code error just now
for index, sentence in tqdm(enumerate(depression_score_api_long_df.text_cleaned.tolist())):
    if depression_score_api_long_df.depression_score[index] == -1.0000:
        score = get_depression_score_from_api(sentence)
        depression_score_api_long_df.depression_score[index] = score

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
19it [00:00, 19.63it/s]

status code not 200!


95it [00:03, 32.87it/s]

status code not 200!


149it [00:04, 36.76it/s]

...

status code not 200!


1409it [00:51, 45.88it/s]

status code not 200!


1436it [00:52, 27.37it/s]

status code not 200!


## Export the dataset

In [ ]:
depression_score_api_short_df.to_csv(DATA_PATH+'emotion_intensity_sentic_depression_identification_short_data.csv', index=False)
depression_score_api_long_df.to_csv(DATA_PATH+'emotion_intensity_sentic_depression_identification_long_data.csv', index=False)

## Filling up the negative data with a zero

In [ ]:
score_short = pd.read_csv(DATA_PATH+'emotion_intensity_sentic_depression_identification_short_data.csv')
score_long = pd.read_csv(DATA_PATH+'emotion_intensity_sentic_depression_identification_long_data.csv')

In [ ]:
# fill the -1 with zeros for the short data
for index, sentence in tqdm(enumerate(score_short.text_cleaned.tolist())):
    if score_short.depression_score[index] == -1.0000:
        score_short.depression_score[index] = 0.0000

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
834it [00:00, 46711.66it/s]


In [ ]:
score_short.head(50)

,text_cleaned,depression_score
0,I get to spend New Year is home again alone an...,0.44333
1,"Depressed and lonely Stuck in a deep, never en...",0.80000
2,Learning to pretend to have a good time had be...,0.11000
3,So far he stop texting me after I said somethi...,0.25000
4,sigh ?? I have not cried so much I am in so mu...,0.44333
5,Thank god the last presentation is over! tomor...,0.00000
6,"No, I am not wouldepressed because of the weat...",0.66667
7,Very sad day in Edmonton! Praying for all who ...,1.00000
8,Losing my uncle after the wedding it just..I d...,0.33333
9,Nobody care in real life and nobody care here....,0.42571


In [ ]:
# fill the -1 with zeros for the long data
for index, sentence in tqdm(enumerate(score_long.text_cleaned.tolist())):
    if score_long.depression_score[index] == -1.0000:
        score_long.depression_score[index] = 0.0000

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
1436it [00:00, 68200.07it/s]


In [ ]:
score_long.head(50)

,text_cleaned,depression_score
0,Just another night. Another night of feeling l...,0.37600
1,Is it possible to fake depression? I have been...,0.47260
2,Imagine being attractive Imagine what it would...,0.66000
3,"Best moment to have anxiety It is am, I am tir...",0.16298
4,"hi, I am a year-old male from the uk, over the...",0.51067
5,I do not want to die feeling like I wa never l...,0.38480
6,I wa just curious what your feeling are on lon...,0.47901
7,"My hearing, attentiveness and social awareness...",0.15250
8,Hello out there.. Ugh. I battle with depressio...,0.38806
9,Currently waiting at the hospital to be seen I...,0.31188


In [ ]:
# export the final datasets
score_short.to_csv(DATA_PATH+'emotion_intensity_sentic_depression_identification_short_data.csv', index=False)
score_long.to_csv(DATA_PATH+'emotion_intensity_sentic_depression_identification_long_data.csv', index=False)